Copyright (C) 2022  Nguyễn Ngọc Sáng

[@SangVn](https://github.com/SangVn) [@VnCFD](https://vncfdgroup.wordpress.com/)

*Thực hành CFD với Python!* 

# Phương pháp lưới Boltzmann (Lattice Boltzmann Method)

Cùng với các phương pháp CFD cổ điển, phương pháp lưới Boltzmann (LBM) ngày càng được phát triển và sử dụng rộng rãi trong những năm gần đây để mô phỏng động lực lọc chất lỏng và chất khí. Trong phương pháp này thay vì dùng phương trình Navier-Stockes mô tả trạng thái của dòng chảy (đặc trưng bởi các thông số vĩ mô: khối lượng riêng, vận tốc, áp suất, nhiệt độ), phương trình Boltzmann được sử dụng để mô tả chuyển động vi mô của các phân tử lưu chất trong sự tương tác va chạm qua lại với các phân tử xung quanh, thông qua đó để xác định các thông số vĩ mô.
Điểm cộng của LBM bao gồm: thuật toán đơn giản, có khả năng thiết lập các điều kiện biên phức tạp, dễ áp dụng tính toán song song. Cùng với đó điểm trừ xuất hiện khi vận tốc dòng chảy lớn (M>0.3), tuy nhiên trong trường hợp này nhiều phương pháp giải quyết đã được đề xuất. Ngoài ra, LBM cần rất nhiều bộ nhớ để lưu trữ các hàm phân bố.


## 1. Phương trình Boltzmann

Bỏ qua ngoại lực tác dụng lên các phân tử, **phương trình Boltzmann** có dạng [1, 2]:
\begin{equation}
\frac{\partial f}{\partial t} + \mathbf{v} \nabla f = Q,
\tag{1}
\end{equation}
trong đó $f(\mathbf{x}, \mathbf{v},t)$ - hàm phân bố (mật độ xác suất) của hạt chuyển động với vận tốc $\mathbf{v}$ tại tọa độ $\mathbf{x}$ và thời điểm $t$, $Q$ - toán tử va chạm (collision integral) đặc trưng cho tốc độ thay đổi của hàm phân bố do va chạm giữa các hạt. 

Các thông số vĩ mô của dòng chảy được xác định theo các công thức:
\begin{equation}
\rho(\mathbf{x}, t) = \int f(\mathbf{x}, \mathbf{v},t) d\mathbf{v}, \\
\rho(\mathbf{x}, t) \mathbf{u}(\mathbf{x},t) = \int \mathbf{v} f(\mathbf{x}, \mathbf{v},t) d\mathbf{v}, \\
\rho(\mathbf{x}, t) \frac{3kT(x,t)}{2m} = \int \frac{|\mathbf{v - u}(\mathbf{x},t)|^2}{2} f(\mathbf{x}, \mathbf{v},t) d\mathbf{v},\\
p=\rho R T,
\tag{2}
\end{equation}
với $k=1.380649×10^{−23} J/K$ - hằng số Boltzmann, $m$ - khối lượng hạt (phân tử khí), $R=k/m $ - hằng số khí đặc trưng (individual/specific gas constant).

Sử dụng **mô hình va chạm Bhatnagar-Gross-Krook (BGK)** [1] với một (đơn) thời gian nghỉ (single relaxation time) ta có:
\begin{equation}
Q = -\frac{1}{ \tau}(f-f^{eq}),
\tag{3}
\end{equation}
với $ \tau$ - thời gian nghỉ (thời gian để hệ đạt đến trạng thái cân bằng nhiệt động cục bộ - the local thermodynamic equilibrium, tức là thời gian để hàm phân bố $f$ trở thành $f^{eq}$), $f^{eq}$ - **hàm phân bố Maxwell-Boltzmann** ở trạng thái cân bằng trong không gian D chiều :
\begin{equation}
f^{eq} = \rho \left(\frac{m}{2\pi k T} \right)^{D/2}exp \left(-\frac{m}{2kT}(\mathbf{v-u})^2 \right).
\tag{4}
\end{equation}
Vận tốc chuyển động của hạt được chia ra làm hai thành phần gồm vận tốc vĩ mô của dòng và vận tốc chuyển động nhiệt. Trong công thức trên, $\mathbf{v-u}$ chính là vận tốc chuyển động nhiệt. Ngoài mô hình va chạm BGK với một thời gian nghỉ, còn có mô hình va chạm nhiều (đa) thời gian nghỉ (multiple relaxation time).

Công thức (4) có thể viết lại ở dạng:
$$f^{eq} = \frac{\rho}{(2\pi \theta)^{D/2}} exp \left(-\frac{(\mathbf{v-u})^2}{2\theta} \right),$$
với $\theta=\frac{kT}{m} = RT$. Hay ta có:
\begin{equation}
f^{eq} = \frac{\rho}{(2\pi \theta)^{D/2}} exp \left(-\frac{\mathbf{v}^2}{2\theta} \right)exp \left(-\frac{\mathbf{u}^2}{2\theta} + \frac{\mathbf{v\cdot u}}{\theta} \right).
\tag{5}
\end{equation}
Đối với khí lý tưởng, vận tốc âm thanh được tính theo công thức $c_s=\sqrt{\gamma R T}= \sqrt{\gamma \theta}$. Đối với quá trình đẳng nhiệt ($T=const$) thì $c_s=\sqrt{\theta}$ hay $\theta = c_s^2, p=\rho c_s^2$. Với dòng chảy có số Mach $M=u/c_s$ nhỏ, sử dụng khai triển Taylor cho công thức (5) thu được:
\begin{equation}
f^{eq} = \frac{\rho}{(2\pi \theta)^{D/2}} exp \left(-\frac{\mathbf{v}^2}{2\theta} \right)\left(1 + \frac{\mathbf{v \cdot u}}{\theta} + \frac{(\mathbf{v \cdot u})^2}{2\theta^2} - \frac{\mathbf{u}^2}{2\theta} \right).
\tag{6}
\end{equation}


## 2. Phương trình lưới Boltzmann
Chia miền không gian thành các ô lưới. Gỉa sử rằng, trong một bước thời gian các hạt chỉ có thể có khả năng di chuyển từ một mắt lưới sang các mắt lưới lân cận theo các hướng cố định, cũng có nghĩa rằng với vận tốc cố định. Ta có thể giả thiết như vậy vì trong chuyển động nhiệt, hạt có thể có vận tốc bất kì (cả độ lớn và hướng). Như ở hình dưới đây thể hiện mô hình D2Q9 ô lưới hình vuông với 9 vận tốc trong không gian hai chiều và D3Q19 ô lưới hình lập phương với 19 vận tốc trong không gian ba chiều. Hai mô hình này thường được sử dụng đủ để đảm bảo độ chính xác và tính ổn định của phương pháp.

<img src='VnCFD_LBM/result/DnQm.png' width='600'>

Khi đó, ta có thể viết lại **phương trình Boltzmann ở dạng rời rạc** [2]:
\begin{equation}
\frac{\partial f_i}{\partial t} + \mathbf{v_i} \nabla f_i = -\frac{1}{\tau}(f_i-f_i^{eq}),
\tag{7}
\end{equation}
$\mathbf{v_i}$ - vận tốc hướng $i$, $f_i=f(\mathbf{x}, \mathbf{v_i}, t)$, $ f_i^{eq}=f^{eq}(\mathbf{x}, \mathbf{v_i}, t)$. Phương trình rời rạc Boltzmann được chuyển về **dạng không thứ nguyên** sử dụng chiều dài đặc trưng $L$, vận tốc tham chiếu $U$ (reference speed), mật độ hạt tham chiếu $n_r$ (reference density) và thời gian giữa các lần va chạm $t_c$
\begin{equation}
\frac{\partial F_i}{\partial \hat t} + \mathbf{c_i} \hat \nabla F_i = -\frac{1}{\hat \tau \epsilon} (F_i - F_i^{eq})
\tag{8}
\end{equation}
với $\mathbf{c_i}=\mathbf{v_i}/U, \hat \nabla = L \nabla, \hat t = tU/L, \hat{\tau} =  \tau/t_c, F_i=f_i/n_r.$ Thông số $$\epsilon = t_c \frac{U}{L}$$ có thể được xem như tỉ lệ giữ thời gian va chạm (collision time) và thời gian dịch chuyển (flow time) hoặc là giữa quãng đường tự do (free path - không có va chạm) và chiều dài đặc trưng (tức là số Knudsen). **Công thức xấp xỉ** của phương trình (8) sẽ là:

$$\frac{F_i(\hat{\mathbf{x}}, \hat t + \Delta \hat t) - F_i(\hat{\mathbf{x}}, \hat t)}{\Delta \hat t} + \mathbf{c_i} \frac{F_i(\hat{\mathbf{x}} + \Delta \hat{\mathbf{x}}, \hat t + \Delta \hat t) - F_i(\hat{\mathbf{x}}, \hat t + \Delta \hat t)}{\Delta \hat {\mathbf{x}}} = -\frac{1}{\hat \tau \epsilon} (F_i - F_i^{eq})$$
với $\Delta \hat t = \Delta t \cdot U/L$. Chọn bước không gian và thời gian sao cho $\Delta \mathbf{\hat x}/\Delta \hat t = \mathbf{c_i}$, ta có:
\begin{equation}
\frac{F_i(\hat{\mathbf{x}} + \mathbf{c_i}\Delta \hat t, \hat t + \Delta \hat t) - F_i(\hat{\mathbf{x}}, \hat t)}{\Delta \hat t}  = -\frac{1}{\hat \tau \epsilon} (F_i - F_i^{eq}).
\tag{9}
\end{equation}
Chọn $\Delta t = t_c$, nhân hai vế phương trình trên cho $\Delta \hat t$ và sau đó bỏ đi kí hiệu mũ $(\hat{})$, thay $F$ bằng $f$ thu được **phương trình lưới (BGK) Boltzmann:**
\begin{equation}
f_i(\mathbf x + \mathbf c_i \Delta t, t+\Delta t) - f_i(\mathbf x, t) = -\frac{1}{\tau}(f_i - f_i^{eq}).
\tag{10}
\end{equation}
Ta viết lại phương trình trên ở dạng:
\begin{equation}
f_i(\mathbf x + \mathbf c_i \Delta t, t+\Delta t) = f_i(\mathbf x, t) -\frac{1}{\tau}(f_i - f_i^{eq}).
\tag{11}
\end{equation}
Ở đây $f_i(\mathbf x + \mathbf c_i \Delta t, t+\Delta t)$ chính là giá trị hàm phân bố sau một bước thời gian ở điểm lưới kế cận theo hướng vận tốc $\mathbf{c_i}$. Để tính giá trị của nó, người ta thường chia ra làm hai giai đoạn:
1. **Va chạm** (collision): $$f_i(\mathbf{x}, t+\Delta t) = f_i(\mathbf{x},t) -\frac{1}{\tau}(f_i - f_i^{eq}) $$

2. **Dịch chuyển** (streaming): $$f_i(\mathbf{x} + \mathbf{c_i}\Delta t, t+\Delta t)=f_i(\mathbf{x}, t+\Delta t)$$

Ta có bảng tóm tắt:
<img src='VnCFD_LBM/result/LB_equations.png' width='600'>

Với mô hình D2Q9, các vận tốc $\mathbf{c_i}=c \mathbf{e_i}$, với $c=\Delta x/\Delta t$ và
\begin{equation}
\mathbf{e_0} = (0, 0), \mathbf{e_1} = (1, 0), \mathbf{e_2} = (-1, 0), \mathbf{e_3} = (0, 1), \mathbf{e_4} = (0, -1), \\ \mathbf{e_5} = (1, 1), \mathbf{e_6} = (-1, 1), \mathbf{e_7} = (-1, -1), \mathbf{e_8} = (1, -1).
\end{equation}

Trong "hệ đơn vị lưới" không thứ nguyên chọn bước thời gian và bước không gian bằng 1 đơn vị: $\Delta x = \Delta t = 1$, khi đó  $c=1$. Các thông số vĩ mô như khối lượng riêng, vận tốc được tính theo công thức:
\begin{equation}
\rho = \sum_i{f_i}, ~ u = \frac{1}{\rho} \sum_i{\mathbf{e_i}f_i}.
\tag{12}
\end{equation}
Trong hệ đơn vị này, vận tốc âm thanh cho mô hình D2D9 (D3Q19) là $c_s=1/\sqrt{3}$, áp suất hay phương trình trạng thái tương ứng là $p=\rho c_s^2$. Độ nhớt động học $\nu=\frac{1}{3}(\tau - \frac{1}{2})$.

Từ phương trình (6) ta có công thức xấp xỉ cho hàm phân bố cân bằng:
\begin{equation}
f_i^{eq}=\rho w_i \left(1 + \frac{\mathbf{e_i \cdot u}}{c_s^2} + \frac{(\mathbf{e_i \cdot u})^2}{2c_s^4} - \frac{\mathbf{u}^2}{2c_s^2} \right),
\tag{13}
\end{equation}

với $w_i$ là các trọng số phụ thuộc vào từng loại lưới. Các trọng số   này đặc trưng cho khả năng (xác suất) hạt có các vận tốc $\mathbf{e_i}$ tương ứng. Nhìn vào hàm phân bố trong công thức (6)
$$f(\mathbf{v}) = \frac{1}{(2\pi\theta)^{D/2}}exp\left(-\frac{\mathbf{v}^2}{2\theta} \right)$$
ta thấy giá trị $f$ không phụ thuộc vào hướng của vận tốc, độ lớn vận tốc càng nhỏ thì $f$ càng lớn, xác suất hạt đứng yên (có vận tốc bằng không) là lớn nhất. Tính toán cho thấy, trọng số tối ưu đối với lưới D2Q9 là:
$$w_0=4/9, w_1=...=w_4=1/9, w_5=...=w_8=1/36,$$
và với lưới D3Q19:
$$w_0=1/3, w_1=...=w_6=1/18, w_7=...=w_{18}=1/36,$$
chú ý: $\sum_i{w_i}=1.$

**Thuật toán LBM:**
1. Tại thời điểm $t=0$, thiết lập điều kiện ban đầu cho các hàm phân bố $f_i$. Ví dụ, nếu $\rho_0, \mathbf{u_0}$ đã biết thì dùng công thức (13) để xác định $f_i=f_i^{eq}$.
2. Tại mỗi điểm lưới xác định các thông số vĩ mô theo công thức (12), thiết lập điều kiện biên cho thông số vĩ mô.
3. Dùng công thức (13) xác định $f_i^{eq}$.
4. Dùng công thức (11) xác định giá trị mới của $f_i$: $f_{i,new} = f_i -1/\tau(f_i-f_i^{eq})$, thiết lập điều kiện biên cho hàm phân bố.
5. Sau một bước thời gian, các hạt sẽ dịch chuyển từ điểm lưới này sang điểm lưới khác theo hướng vận tốc $\mathbf{e_i}$, tức là giá trị $f_{i,new}$ cũng dịch chuyển tới vị trí tưng tự $f_i(\mathbf{x+e_i}\Delta t, t+\Delta t) = f_{i,new}$.
6. Lặp lại các bước từ 2 tới 5.

# 3. Điều kiện biên (boundary conditions - BC)

Xét điều kiện biên cho mô hình D2Q9. Trên biên trái (inlet) như ở hình dưới, ta thấy sau dịch chuyển các hàm phân bố $f_{0,2,3,4,6,7}$ đã biết, còn $f_{1,5,8}$ chưa biết. Còn đối với biên phải (outlet), sau dịch chuyển ác hàm phân bố $f_{0,1,3,4,5,8}$ đã biết, còn $f_{2,7,6}$ chưa biết. Nhiệm vụ của điều kiện biên chính là xác định giá trị của các hàm phân bố chưa biết này [2, 3, 4].
<img src='VnCFD_LBM/result/bc.png'>
    

**3.1. Periodic BC** - Điều kiện biên tuần hoàn cho biên trái (inlet) và phải (outlet):
$$f_n(\mathbf{x}_{in}, t) = f_n(\mathbf{x}_{out}, t),\\ f_{\bar n}(\mathbf{x}_{out}, t) = f_{\bar n}(\mathbf{x}_{in}, t),\\ n=[1,5,8], \bar n=[2,7,6]$$

**3.2. Equilibrium BC (first order approach)**

a) Inlet, $\mathbf{u}_{in}$ không đổi:
$$f_n(\mathbf{x}_{in}) = f_n^{eq}(\rho(\mathbf{x}_{in}+\Delta\mathbf x), \mathbf u_{in})$$

b) Outlet, $\rho_{out}$ không đổi:
$$f_{\bar n}(\mathbf{x}_{out}) = f_{\bar n}^{eq}(\rho_{out}, u(\mathbf{x}_{out}-\Delta\mathbf x))$$

**3.3. Zou-He BC**

a) Inlet, $\mathbf{u}_{in}=(u_x, u_y)$ không đổi:
$$f_n(\mathbf{x}_{in}) = f_n^{eq}(\rho(\mathbf{u}_{in}), \mathbf{u}_{in}) + (f_{\bar n} - f_{\bar n}^{eq})$$
Khối lượng riêng $\rho(\mathbf{u}_{in})$ được tính như sau. Từ phương trình (12) ta có: $$\rho = \sum\limits_{i=[2,7,6]} f_i + \sum\limits_{i=[0,3,4]} f_i + \sum\limits_{i=[1,5,8]} f_i = \rho^- + \rho^0 + \rho^+,$$
với $\rho^-, \rho^0$ đã biết, $\rho^+$ chưa biết. Cũng từ phương trình (12) ta lại có: $$\rho u_x = -\rho^- + \rho^+.$$
Loại bỏ $\rho^-$ từ hai phương trình trên ta được:
$$\rho_{in} = \frac{1}{1-u_x}(2\rho^- + \rho^0)$$

b) Outlet, $\rho_{out}$ (hay $p_{out}=\rho_{out}c_s^2$) không đổi:
$$f_{\bar n}(\mathbf{x}_{out}) = f_{\bar n}^{eq}(\rho_{out}, \mathbf u(\rho_{out})) + (f_{n} - f_{n}^{eq})$$
Vận tốc $\mathbf u(\rho_{out})$ được tính như sau. Từ phương trình (12) ta có:
$$\rho = \rho^- + \rho^0 + \rho^+ \\ 
\rho u_x = -\rho^- + \rho^+,\\
\rho u_y = \sum\limits_{i=[3,5,6]} f_i - \sum\limits_{i=[4,7,8]} f_i.$$
Với $\rho^0, \rho^+$ đã biết ta có $\rho^- = \rho - \rho^0 - \rho^+$, và
$$u_{x,out} = \frac{2\rho^+ + \rho^0 - \rho}{\rho}.$$
Ta thấy giá trị $u_{y,out}$ không thể xác định nếu chỉ biết $\rho_{out}$, do đó nó cũng cần phải cho trước.

**3.4. Bounce-back method**

a) No-slip BC ($\mathbf{u_w}=0$) - Các hạt sau khi va chạm với tường rắn $\mathbf{x}_w$ (solid wall) sẽ bay ngược lại vào vùng tính toán:
$$f_i(\mathbf{x}_w, t+\Delta t) = f_{\bar i}(\mathbf{x}_w,t),\\ i=[0,1,2,3,4,5,6,7,8], \bar i= [0,2,1,4,3,7,8,5,6]$$
với $\bar i$ là hướng ngược lại của $i$.

b) Moving wall ($\mathbf{u_w} \ne 0$)
$$f_i(\mathbf{x}_w, t+\Delta t) = f_{\bar i}(\mathbf{x}_w,t) - 2 w_i \rho_0 c_s^2(\mathbf{e_i u_w})$$


**3.5. Yu BC**

a) Inlet ($(\rho_0 \cdot \mathbf{u_0})$ không đổi):
$$f_i(\mathbf{x}_{in})=f_i^{eq}(\rho_0, \mathbf{u_0}) + (f_{\bar i} - f_{\bar i}^{eq})$$
b) Outlet ($\partial_t f_i + \mathbf{e_i}\nabla f_i = 0$):
$$f_i(\mathbf{x}_{out}) = f_i(\mathbf{x}_{out}-\mathbf{e_i}\Delta t)$$

# 4. 2D Code

Vì thuật toán đơn giản nên ta có một chương trình khá gọn nhẹ cho việc mô phỏng dòng chảy hai chiều bằng phương pháp lưới Boltzmann sử dụng ngôn ngữ lập trình python. Các thư viện cần thiết bao gồm: *numpy, matplotlib, tqdm*.

**4.1. Thư mục lib**

Trong thư mục **lib** có 3 modules **lbData, lbBoco, lbSolver**. 

Module **lbData** chứa class **lbNodes** với các hàm cơ bản: khởi tạo (*__ init__*), thiết lập điều kiện ban đầu (*init*), thiết lập điều kiện biên (*set_boco*), tính các thông số macro (*macro_vars*), xác định hàm phân bố cân bằng (*equilibrium*), tính hàm phân bố sau va chạm (*collision*), áp dụng điều kiện biên bounce back cho vật cản (obstacle) nếu có (*bounce_back*), thực hiện bước dịch chuyển (*streaming*), thực hiện một bước lặp (*iteration*).

Module **lbBoco** chứa ba loại hàm: xác định các thông số macro trên mỗi biên, xác định hàm phân bố trên mỗi biên, tổng hợp xác định hàm phân bố trên cả 4 biên miền 2D.

Module **lbSolver** chứa hàm *show_field* để vẽ các trường vận tốc, khối lượng riêng, xoáy nếu cần, và hàm *solver* thực hiện các bước lặp thời gian, lưu kết quả vào thư mục *result*. Hàm solver có các tham số *solver(Nx, Ny, rho0, u0, tau, Nt, boco_macro, boco_f, obstacle=None, plot_field='velocity', plot_obs=None, fname='result.png')*, với *Nx, Ny* -- số điểm lưới trên mỗi chiều, *rho0, u0* -- điều kiện ban đầu, *tau* -- bước thời gian, *Nt* -- số bước lặp thời gian, *boco_macro, boco_f* -- các hàm xác định thông số macro và hàm phân bố trên biên, *obstacle* -- vật thể trong dòng chảy (nếu có), *plot_field* -- tên trường thông số để vẽ và hiển thị (nếu cần), *plot_obs* -- hàm vẽ vật cản (nếu cần), *fname* -- tên file hình ảnh khi lưu kết quả.

**4.2. Ví dụ**

Có sáu ví dụ được xem xét đó là các dòng chảy poiseuille, lid driven cavity, cylinder, naca23012, dòng chảy qua khuôn mặt một cô gái và một chú cá voi. Mỗi ví dụ tương ứng với một module cùng tên. Trong mỗi module có các hàm chính gồm: hàm xác định thông số macro trên biên, hàm xác định vật cản, hàm xử lý kết quả và hàm *main* nơi thiết lập các thông số mô phỏng. Có hai cách để cho *rho0, u0* ban đầu:

a) cho các giá trị xác định, ví dụ: *rho0, u0 = 1.0, [0.0, 0.0]*

b) lấy kết quả từ lần tính trước để tiếp tục: 
*rho0 = np.load('result/rho.npy'), u0 = [np.load('result/ux.npy'), np.load('result/uy.npy')]*
  
**a) Dòng chảy Poiseuille**

Dòng chảy tầng trong kênh với chênh lệch áp suất ở đầu vào và đầu ra $dp$, profile vận tốc được xác định theo công thức
$$u(x,y,t) = dp~ y(H-y)/(2\mu L)$$
với $L,H$ là chiều dài và rộng của kênh. Ta thiết lập điều kiên biên áp suất cho inlet vào outlet, điều kiện biên vận tốc ($u=[0,0]$) cho thành kênh. Hình dưới dây so sánh kết quả vận tốc và nghiệm chính xác.

<img src='VnCFD_LBM/result/poiseuille_velocity.png' width=400>

**b) Lid driven cavity**

Dòng chảy trong hốc với vận tốc trên bề mặt thoáng cố định. Sử dụng điều kiện biên vận tốc cho cả 4 biên. Trường vận tốc và đường dòng cho hai trường hợp $Re=100, Re=1000$ được biểu diễn ở hình dưới.
<img src='VnCFD_LBM/result/cavity.png' width=600>

**c) Cylinder**

Dòng chảy quanh hình trụ ở trong kênh. Vận tốc ở đầu vào được thiết lập bởi profile vận tốc dòng chảy Poiseuille, ở đầu ra dùng điều kiện biên mở $f_i(x_{out}) = f_i(x_{out}-\Delta x)$. Xét hai trường hợp $Re=20, Re=100$.

$Re = 20$, streamline:
<img src='VnCFD_LBM/result/cylinder_streamline_Re20.png' width=500>
$Re = 100$, vorticity:
<img src='VnCFD_LBM/result/cylinder_vorticity_Re100.gif' width=600>

**d) NACA23012**

Dòng chảy quanh NACA23012 ở trong kênh, $Re=400$.

Streamline:
<img src='VnCFD_LBM/result/naca23012_streamline.png' width=300>
Vorticity:
<img src='VnCFD_LBM/result/naca23012_vorticity.gif' width=500>

**e) face_whale**

Face: những dòng xoáy được tạo ra như mái tóc dài của cô gái đang tung bay trong gió.
<img src='VnCFD_LBM/result/face.gif' width=600>
Whale:
<img src='VnCFD_LBM/result/whale.png' width=600>

# 5. Kết luận

Trong bài viết này đã giới thiệu các kiến thức cơ bản về phương pháp lưới Boltzmann, tuy nhiều còn nhiều vấn đề mà bạn đọc cần tìm hiểu thêm, trong số đó có:

1. Cách chuyển đổi từ các thông số vĩ mô có đơn vị sang các thông số không đơn vị LBM và ngược lại.
2. Điều kiện ổn định của phương pháp.
3. Chính xác hóa điều kiện biên trên bề mặt vật cản, trong bài viết không đề cập tới việc các điểm lưới bề mặt có thể không nằm trên bề mặt của vật.
4. Áp dụng tính toán song song để tăng tốc độ mô phỏng.
5. Mô phỏng dòng chảy có số Mach lớn.

Với những điểm cộng của mình, phương pháp lưới Boltzmann ngày càng được sử dụng rộng rãi để mô phỏng các bài toán khác nhau của chuyển động chất lưu, từ các chuyển động thông thường, các dòng đa pha, cho tới chuyển động trong môi trường vật liệu xốp, nanofluid, magnetic fluid... Do đó, đây là một mảnh đất màu mỡ mà nếu bạn đọc bỏ công sức đào bới, vun trồng sẽ có ngày thu về quả ngọt.

Bài viết này được hoàn thành trong một thời gian ngắn. Nó là kết quả khi tác giả bị kích thích bởi cái mới, sự đơn giản của LBM trong thuật toán nên muốn tìm hiểu và giới thiệu tới bạn đọc. Tác giả không tìm hiểu sâu về phương pháp này. Do đó nếu có sai sót, mong bạn đọc góp ý. Sẽ rất vui nếu được trao đổi với các bạn, đặc biệt là những bạn đang nghiên cứu, sử dụng phương pháp LBM.

Chúc bạn đọc sức khỏe và thành công!

##  Tài liệu tham khảo

1. 1954, *P.L Bhatnagar, E.P. Gross, M. Krook*, A Model for Collision Processes in Gases. I. Small Amplitude Processes in Charged and Neutral One-Component Systems.
2. 2000, *Dieter A. Wolf Gladrow*, Lattice-Gas Cellular Automata and Lattice Boltmann Models.
3. 2008, *Salvador Izquierdo Estallo*, Compuatational Gas Dynamics with the Lattice Boltzmann Method: Preconditioning and Boundary Conditions.
4. 2011, *Yuanxun Bill Bao, Justin Meskas*, Lattice Boltzmann Method for Fluid Simulations.
5. 2017, *Timm Kruger, Halim Kusumaatmaja, Alexandr Kuzmin, Orest Shardt, Gocanlo Silva, Erlend Magnus Viggen*, The Lattice Boltzmann Method: Principles and Practice.

Zhukovsky, 05.2022